In [1]:
import pandas as pd
from collections import defaultdict
import tqdm
from sklearn.metrics import jaccard_score
import numpy as np
from collections import Counter

In [2]:
tqdm.tqdm.pandas()

In [3]:
df1 = pd.read_csv('24.csv')
df2 =pd.read_csv('lenta.csv')
df3 = pd.read_csv('vesti.csv')
df = pd.concat([df1,df2,df3])
del df1
del df2
del df3

In [4]:
df = df.dropna()

In [5]:
from string import ascii_lowercase, digits, whitespace
cyrillic = u"абвгдеёжзийклмнопрстуфхцчшщъыьэюя"
allowed_characters = ascii_lowercase + digits + cyrillic + whitespace

In [6]:
def clean(text):
    return ' '.join(''.join([character if character in set(allowed_characters) else ' ' for character in text.lower()]).split()) 

In [7]:
df['cleaned'] = df.text.progress_apply(lambda x: clean(str(x)))

100%|██████████| 8228/8228 [00:41<00:00, 200.64it/s]


In [8]:
def build_shingles(text, n):
    text = text.split()
    shingles = []
    for i in range(0, len(text) - n + 1):
        shingle = []
        for j in range(0, n):
            shingle.append(text[i + j])
        shingles.append(' '.join(shingle))
    return shingles

In [9]:
df['shingles_2'] = df.cleaned.progress_apply(lambda x: build_shingles(x, 2))

100%|██████████| 8228/8228 [00:01<00:00, 5541.76it/s]


In [10]:
BOS = []
for shingle in df['shingles_2']:
    BOS.extend(shingle)

In [11]:
print(len(BOS))
BOS = list(set(BOS))
print(len(BOS))

2325705
706973


In [12]:
bag_dict = {}
for i in range(0, len(BOS)):
    bag_dict[BOS[i]] = i
r = [0] * len(BOS)

In [13]:
arr = np.zeros((df.shape[0], len(BOS)), dtype=np.uint8)
for num, sh in enumerate(df.shingles_2.tolist()):
    for shingle in sh:
        arr[num, bag_dict[shingle]] = 1


In [14]:
arr.shape

(8228, 706973)

In [15]:
index = {}
for shingle in BOS:
    index[shingle] = []

i = 0
for shingles in np.array(df.shingles_2):
    for shingle in shingles:
        index[shingle].append(i)
    i += 1

In [16]:
for i in index.values():
    if len(i) < 1:
        print(i)

In [30]:
def count_sim(shingles):
    result = []

    for sh in shingles:
        result.extend(index[sh])
    result_count = Counter(result)
    try:
        minimum = np.max(list(result_count.values())) * 0.5
    except:
        minimum=0
        # print(shingles)
    result = []

    for key, value in result_count.items():
        if value > minimum:
            result.append(key)

    return len(result) - 1

In [18]:
def get_sim(i):
    result = []
    for sh in np.array(df[df['Unnamed: 0'] == i].shingles_2)[0]:
        result.extend(index[sh])
    result_count = Counter(result)
    minimum = np.max(list(result_count.values())) * 0.5
    result = []
    for key, value in result_count.items():
        if value > minimum and key != i:
            result.append(key)
    sim = {}
    v1 = arr[i]
    for i in result:
        v2 = arr[i]
        sim[i] = jaccard_score(v1, v2, average='micro')

    return sim

In [31]:
df['count_sim'] = df.shingles_2.progress_apply(count_sim)
co_sim = df[df.count_sim > 0]
co_sim_1 = co_sim[co_sim.count_sim == 1]


 15%|█▌        | 1252/8228 [00:02<00:14, 496.53it/s]

KeyboardInterrupt: 

In [20]:
co_sim_1['sim'] = co_sim_1['Unnamed: 0'].progress_apply(get_sim)

100%|██████████| 341/341 [01:07<00:00,  5.00it/s]
/usr/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
co_sim_1['sim_value'] = co_sim_1.sim.apply(lambda x: list(x.values()))
co_sim_1 = co_sim_1.sort_values(['sim_value'], ascending=False)

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [22]:
co_sim_1

,Unnamed: 0,date,url,title,text,cleaned,shingles_2,count_sim,sim,sim_value
4353,4353,2019/04/21,https://lenta.ru/news/2019/04/21/dmbykov/,Дмитрий Быков вновь заговорил,Писатель Дмитрий Быков пришел в сознание и мож...,писатель дмитрий быков пришел в сознание и мож...,"[писатель дмитрий, дмитрий быков, быков пришел...",1,"{5245: 0.9997510822970013, 5220: 0.99962664667...","[0.9997510822970013, 0.9996266466790646]"
905,905,2019/04/23,https://24smi.org/news/152083-uchenye-rasskaza...,Ученые рассказали о способе избежать ранней см...,Сидячий образ жизни повышает риск преждевремен...,сидячий образ жизни повышает риск преждевремен...,"[сидячий образ, образ жизни, жизни повышает, п...",1,{4764: 0.9996294745893107},[0.9996294745893107]
4204,4204,2019/04/22,https://lenta.ru/news/2019/04/22/big_badaboom/,Опубликовано видео нового взрыва в Шри-Ланке,Видео подрыва взрывного устройства в микроавто...,видео подрыва взрывного устройства в микроавто...,"[видео подрыва, подрыва взрывного, взрывного у...",1,"{5096: 0.9995248472369725, 5092: 0.99941740351...","[0.9995248472369725, 0.9994174035145275]"
3225,3225,2019/04/26,https://lenta.ru/news/2019/04/26/sredmore/,Российские военные отказались беспокоиться из-...,Россия провела крупномасштабные учения в Среди...,россия провела крупномасштабные учения в среди...,"[россия провела, провела крупномасштабные, кру...",1,"{4134: 0.9994287128362852, 4410: 0.99930997133...","[0.9994287128362852, 0.9993099713383586]"
1947,1947,2019/05/07,https://lenta.ru/news/2019/05/07/su57/,Минобороны получит первый серийный Су-57,Первый российский серийный истребитель пятого ...,первый российский серийный истребитель пятого ...,"[первый российский, российский серийный, серий...",1,"{2880: 0.9994258854938516, 4937: 0.99933541475...","[0.9994258854938516, 0.9993354147577516]"
2631,2631,2019/04/30,https://lenta.ru/news/2019/04/29/come_back/,Дмитрия Быкова выписали из больницы,"Писатель Дмитрий Быков сообщил «Интерфаксу», ч...",писатель дмитрий быков сообщил интерфаксу что ...,"[писатель дмитрий, дмитрий быков, быков сообщи...",1,"{3550: 0.9994032670422408, 4088: 0.99944850445...","[0.9994032670422408, 0.9994485044572152]"
1992,1992,2019/05/07,https://lenta.ru/news/2019/05/07/savchenko/,Суд отказался возвращать Савченко в тюрьму,Броварский межрайонный суд Киевской области от...,броварский межрайонный суд киевской области от...,"[броварский межрайонный, межрайонный суд, суд ...",1,"{2925: 0.9993778218958616, 4188: 0.99923081817...","[0.9993778218958616, 0.9992308181747485]"
3777,3777,2019/04/24,https://lenta.ru/news/2019/04/24/zabota/,Путин ответил на критику упрощенного гражданст...,Упрощение получения гражданства жителям отдель...,упрощение получения гражданства жителям отдель...,"[упрощение получения, получения гражданства, г...",1,"{4674: 0.9993665131498013, 4710: 0.99801321508...","[0.9993665131498013, 0.9980132150871303]"
1352,1352,2019/05/12,https://lenta.ru/news/2019/05/12/head/,Иран пригрозил США «ударом в голову»,В случае активных действий со стороны США в Пе...,в случае активных действий со стороны сша в пе...,"[в случае, случае активных, активных действий,...",1,"{2295: 0.9993636859832749, 1837: 0.99927604778...","[0.9993636859832749, 0.9992760477865019]"
1176,1176,2019/05/13,https://lenta.ru/news/2019/05/13/ssj100/,Кремль уточнил свои полномочия в вопросе запре...,Кремль не имеет полномочий для решения вопроса...,кремль не имеет полномочий для решения вопроса...,"[кремль не, не имеет, имеет полномочий, полном...",1,"{2124: 0.9993467231520183, 2956: 0.99760109179...","[0.9993467231520183, 0.9976010917999159]"


In [23]:
co_sim_1[co_sim_1['Unnamed: 0']==4353].iloc[0].text

'Писатель Дмитрий Быков пришел в сознание и может разговаривать. Об этом сообщает ТАСС.«Он пришел в сознание. Разговаривает и командует. И дышит сам», — сообщил журналистам председатель редакционного совета «Новой газеты» Дмитрий Муратов.Ранее жена писателя рассказала, что Быков «переведен на самостоятельное дыхание».Дмитрия Быкова госпитализировали в Уфе 17 апреля. Ему стало плохо в самолете. Писателя погрузили в медикаментозную кому. Спустя несколько часов Быкова доставили в Москву, специалисты отметили, что состоянию больного ничто не угрожает.Ранее в прессе появлялась информация, что врачи диагностировали у писателя инфаркт головного мозга, однако Муратов это опроверг.'

In [24]:
df.text.iloc[5245]

'Писатель Дмитрий Быков пришел в сознание и может разговаривать. Об этом сообщает ТАСС.«Он пришел в сознание. Разговаривает и командует. И дышит сам», — сообщил журналистам председатель редакционного совета «Новой газеты» Дмитрий Муратов.Ранее жена писателя рассказала, что Быков «переведен на самостоятельное дыхание».Дмитрия Быкова госпитализировали в Уфе 17 апреля. Ему стало плохо в самолете. Писателя погрузили в медикаментозную кому. Спустя несколько часов Быкова доставили в Москву, специалисты отметили, что состоянию больного ничто не угрожает.Ранее в прессе появлялась информация, что врачи диагностировали у писателя инфаркт головного мозга, однако Муратов это опроверг.'

In [25]:
df.text.iloc[5220]

'Писатель и журналист Дмитрий Быков пришел в сознание. Об этом сообщает телеканал «360».Врачи НИИ нейрохирургии имени Бурденко отмечают у него положительную динамику. По словам супруги писателя Ирины Лукьяновой, Быков еще очень слаб и плохо себя чувствует. Ему перестали давать лекарства, обеспечивающие медикаментозный сон, и он начал «просыпаться потихоньку», сообщила жена писателя, добавив, что Быков был снят с аппарата искусственной вентиляции лёгких.20 апреля Лукьянова сообщила о положительной динамике в состоянии супруга — он начал реагировать на внешние раздражители.Дмитрия Быкова госпитализировали в Уфе 17 апреля. Ему стало плохо в самолете. Писателя погрузили в медикаментозную кому. Спустя несколько часов Быкова доставили в Москву, специалисты отметили, что состоянию больного ничего не угрожает.Ранее в прессе появилась информация о том, что врачи диагностировали у писателя инфаркт головного мозга. Однако издатель «Новой газеты» Дмитрий Муратов опроверг эту информацию.'

In [26]:
df.text.iloc[3225]

'В сети появилось видео, на котором запечатлено, как горящий пассажирский самолет Superjet-100 садится в столичном аэропорту Шереметьево. Ролик опубликован в Telegram-канале «112».Ранее сообщалось, что лайнер выполнял рейс из Москвы в Мурманск, однако из-за пожара, который заметил один из пассажиров, совершил экстренную посадку. В настоящее время аэропорт закрыли на вылет и посадку. Все полеты отложены.Сообщается, что в результате инцидента могли пострадать люди, предположительно, шесть человек получили ушибы и, возможно, переломы. Источник РИА Новости рассказал о пятерых пострадавших.'